### Como manipular XML com XPath

In [7]:
from gensim.models import KeyedVectors
import unicodedata

import gzip
import xml.etree.ElementTree as ET

with gzip.open('pordesc2018-small.xml.gz') as pordesc2018:
    tree = ET.parse(pordesc2018)
    

In [8]:
### Monta Dicionario

In [9]:
from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

### Monta Dicionario

In [10]:
wordModel = KeyedVectors.load_word2vec_format('health_w2v_unigram_50.bin', binary=True)

dictMesh = {}
for d in tree.findall("./DescriptorRecord"):
    terms = []
    
    qualifier = '#'
    
    for aql in d.findall('.AllowableQualifiersList/AllowableQualifier/QualifierReferredTo/QualifierName'):
        teste_qualifier = aql.find('./String').text
        if(teste_qualifier == 'anatomy & histology' or teste_qualifier == 'pharmacology' or teste_qualifier == 'methods' or teste_qualifier == 'diagnosis'):
            qualifier = teste_qualifier
            break
    
    for c in d.findall('.ConceptList/'):
        if c.find('./ScopeNote') != None:
            scope = c.find('./ScopeNote').text.replace('\n','').strip()
        for t in c.findall('./TermList/'):
            terms.append(t.find('./String').text)
            
            palavra_similar = []
            
            if t.find('./String').text.lower() in wordModel.vocab:
                sem_assento = remover_acentos(t.find('./String').text)
                palavra_similar = wordModel.most_similar_cosmul(sem_assento.lower(),topn=10)
                for similar, porcentagem in palavra_similar:
                    if(porcentagem > 0.9 and qualifier != 'pharmacology'):
                        terms.append("<i>"+similar+"</i>" + " <input class='radio' type='radio' name='"+similar+"' value='1'/> Certo <input class='radio' type='radio' name='"+similar+"'' value='0'/> Errado")
                    elif(porcentagem > 0.95 and qualifier == 'pharmacology'):
                        terms.append("<i>"+similar+"</i>")
    
    terms.append(" <input type='hidden' class='hidden' name='ID' value='"+d.find('.DescriptorUI').text+"'/> ")                         
    dictMesh[d.find('.DescriptorUI').text] = {
        'ID': d.find('.DescriptorUI').text,
        'name': d.find('.DescriptorName/String').text,
        'scope': scope,
        'terms': sorted(set(terms), reverse=True),
        'qualifier': qualifier
    }
                
    #break
len(dictMesh)

1859

### Salva Dicionario

In [11]:
import gzip, pickle

with gzip.open('dictMesh.dict_eng.gz','wb') as fp:
    pickle.dump(dictMesh,fp)
    fp.close()

### Carrega Dicionario

In [12]:
import gzip, pickle

with gzip.open('dictMesh.dict.gz','rb') as fp:
    dictMesh = pickle.load(fp)
    fp.close()
    
dictMesh['D000001']

{'ID': 'D000001',
 'name': 'Calcimicina[Calcimycin]',
 'scope': 'An ionophorous, polyether antibiotic from Streptomyces chartreusensis. It binds and transports CALCIUM and other divalent cations across membranes and uncouples oxidative phosphorylation while inhibiting ATPase of rat liver mitochondria. The substance is used mostly as a biochemical tool to study the role of divalent cations in various biological systems.',
 'terms': ['Calcimycin',
  'Calcimicina',
  'Antibiotic A23187',
  'A23187, Antibiotic',
  'A23187',
  'A-23187',
  'A 23187',
  " <input type='hidden' name='ID' value='D000001'/> "],
 'qualifier': 'pharmacology'}

In [13]:
palavra = 'Calcimicina'
for dui in dictMesh:
    d = dictMesh[dui]
    for t in d['terms']:
        if t.lower() == palavra.lower():
            print(t)
            print(d['scope'])

Calcimicina
An ionophorous, polyether antibiotic from Streptomyces chartreusensis. It binds and transports CALCIUM and other divalent cations across membranes and uncouples oxidative phosphorylation while inhibiting ATPase of rat liver mitochondria. The substance is used mostly as a biochemical tool to study the role of divalent cations in various biological systems.
